# **ALGORITHMS FOR MASSIVE DATA**
# Market Basket Analysis on IMBD dataset

----

Summary:

1.   Imports
2.   Pre-processing





## **1. Imports**

Importing kaggle.

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"martinaviggiano","key":"53a50dae23b35b3c81b9063b6c417e93"}'}

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing imdb-dataset data.

In [4]:
!kaggle datasets download -d ashirwadsangwan/imdb-dataset

imdb-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip imdb-dataset.zip

Archive:  imdb-dataset.zip
replace name.basics.tsv.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## **2. Pre-processing**

We will consider movies as baskets and actors as items.


In [6]:
!pip install pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [8]:
import pandas as pd
import pickle

In [9]:
title_basics = spark.read.csv('title.basics.tsv.gz', header=True, sep = '\t')

In [10]:
title_basics = title_basics.filter((title_basics.titleType == 'movie'))
title_basics.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000009|    movie|          Miss Jerry|          Miss Jerry|      0|     1894|     \N|            45|             Romance|
|tt0000147|    movie|The Corbett-Fitzs...|The Corbett-Fitzs...|      0|     1897|     \N|            20|Documentary,News,...|
|tt0000335|    movie|Soldiers of the C...|Soldiers of the C...|      0|     1900|     \N|            \N|     Biography,Drama|
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|                  \N|
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Biography,Crim

In [11]:
title_principals = spark.read.csv('title.principals.tsv.gz', header=True, sep = '\t')

In [12]:
title_principals = title_principals.filter((title_principals.category == "actor") | (title_principals.category == "actress"))
title_principals = title_principals.select("tconst","nconst")
title_principals.show(5)

+---------+---------+
|   tconst|   nconst|
+---------+---------+
|tt0000005|nm0443482|
|tt0000005|nm0653042|
|tt0000007|nm0179163|
|tt0000007|nm0183947|
|tt0000008|nm0653028|
+---------+---------+
only showing top 5 rows



In [13]:
df_title = title_principals.join(title_basics, ["tconst"], 'leftsemi')
df_title.show(5)

+---------+---------+
|   tconst|   nconst|
+---------+---------+
|tt0000335|nm1010955|
|tt0000335|nm1012612|
|tt0000335|nm1011210|
|tt0000335|nm1012621|
|tt0000335|nm0675239|
+---------+---------+
only showing top 5 rows



In [14]:
name_basics = spark.read.csv('name.basics.tsv.gz', header=True, sep = '\t')

In [ ]:
#df_title = title_principals[title_principals.set_index(['tconst']).index.isin(title_basics.set_index(['tconst']).index)]

In [ ]:
df_title.write.format('csv').option('header',True).mode('overwrite').option('sep','|').save("/content/drive/MyDrive/Università/amd_Market_Basket/df_title.csv")

In [ ]:
df_title.rdd.getNumPartitions()

In [ ]:
!cp df_title.csv /content/drive/MyDrive/Università/amd_Market_Basket

In [ ]:
#title_basics.to_csv("title_basics.csv", index=False, sep="|")

In [ ]:
#!cp title_basics.csv /content/drive/MyDrive/Università/amd_Market_Basket

In [ ]:
#title_principals.to_csv("title_principals.csv", index=False, sep="|")

In [ ]:
#!cp title_principals.csv /content/drive/MyDrive/Università/amd_Market_Basket

Load back 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_title_ = spark.read.csv("/content/drive/MyDrive/Università/amd_Market_Basket/df_title.csv", header=True, sep = '|')

In [15]:
df_title.summary().show()

+-------+---------+---------+
|summary|   tconst|   nconst|
+-------+---------+---------+
|  count|  1694722|  1694722|
|   mean|     null|     null|
| stddev|     null|     null|
|    min|tt0000009|nm0000001|
|    25%|     null|     null|
|    50%|     null|     null|
|    75%|     null|     null|
|    max|tt9916730|nm9993680|
+-------+---------+---------+



In [ ]:
df_title_.summary().show()

In [16]:
from pyspark.sql.functions import collect_set, col, count

baskets = df_title.groupBy('tconst').agg(collect_set('nconst').alias('actors'))
baskets.createOrReplaceTempView('baskets')
df_title.show(5)
baskets.show(5)

+---------+---------+
|   tconst|   nconst|
+---------+---------+
|tt0000335|nm1010955|
|tt0000335|nm1012612|
|tt0000335|nm1011210|
|tt0000335|nm1012621|
|tt0000335|nm0675239|
+---------+---------+
only showing top 5 rows

+---------+--------------------+
|   tconst|              actors|
+---------+--------------------+
|tt0000335|[nm1012612, nm067...|
|tt0000630|         [nm0624446]|
|tt0000676|[nm0140054, nm009...|
|tt0000793|         [nm0691995]|
|tt0000862|[nm0264569, nm528...|
+---------+--------------------+
only showing top 5 rows



In [17]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="actors", minSupport=0.00001, minConfidence=0.00001)
model = fpGrowth.fit(baskets)

In [18]:
# Display frequent itemsets.
mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")
mostPopularItemInABasket.show()

# Display generated association rules.
associationRules = model.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show()

# transform examines the input items against all the association rules and summarize the consequents as prediction
associations = model.transform(baskets)
associations.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+----+
|               items|freq|
+--------------------+----+
|         [nm0387390]|  20|
|         [nm0084153]|   7|
|         [nm1209934]|   4|
|         [nm0927092]|   9|
|         [nm0271806]|  46|
|[nm0271806, nm015...|   5|
|[nm0271806, nm093...|   7|
|[nm0271806, nm067...|   7|
|[nm0271806, nm047...|   4|
|[nm0271806, nm039...|   5|
|[nm0271806, nm039...|   4|
|[nm0271806, nm051...|   7|
|[nm0271806, nm004...|   4|
|         [nm1171731]|   6|
|         [nm1008709]|   9|
|         [nm0194638]|  33|
|         [nm9562694]|   4|
|         [nm1037451]|   8|
|[nm1037451, nm231...|   4|
|         [nm8920170]|   7|
+--------------------+----+
only showing top 20 rows

+--------------------+-----------+-------------------+------------------+--------------------+
|          antecedent| consequent|         confidence|              lift|             support|
+--------------------+-----------+-------------------+------------------+--------------------+
|         [nm1002

In [19]:
query = """select items, freq
           from mostPopularItemInABasket
           where size(items) > 5
           order by freq desc
           limit 20"""
spark.sql(query).show(5)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[nm2687024, nm237...|  20|
|[nm2426685, nm207...|  17|
|[nm2373151, nm236...|  17|
|[nm2414317, nm207...|  16|
|[nm0455938, nm268...|  15|
+--------------------+----+
only showing top 5 rows



In [20]:
query = """select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
           from associationRules
           order by confidence desc
           limit 20"""

spark.sql(query).show(10)

+--------------------+-----------------+----------+
|     antecedent (if)|consequent (then)|confidence|
+--------------------+-----------------+----------+
|[nm3722581, nm288...|      [nm1990182]|       1.0|
|[nm3808706, nm288...|      [nm1984183]|       1.0|
|[nm7096417, nm040...|      [nm2366585]|       1.0|
|[nm1039432, nm030...|      [nm0042874]|       1.0|
|[nm1283907, nm242...|      [nm0648803]|       1.0|
|[nm1283907, nm242...|      [nm2373718]|       1.0|
|         [nm6500818]|      [nm5771748]|       1.0|
|         [nm3721804]|      [nm1984183]|       1.0|
|[nm2762012, nm236...|      [nm0648803]|       1.0|
|[nm2719915, nm128...|      [nm0648803]|       1.0|
+--------------------+-----------------+----------+
only showing top 10 rows



In [21]:
name_basics.show(5)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00599...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,soun...|tt0078723,tt00804...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [22]:
name_basics = name_basics.select("nconst","primaryName")
name_basics.show(2)

+---------+-------------+
|   nconst|  primaryName|
+---------+-------------+
|nm0000001| Fred Astaire|
|nm0000002|Lauren Bacall|
+---------+-------------+
only showing top 2 rows



In [23]:
df_title.show(2)

+---------+---------+
|   tconst|   nconst|
+---------+---------+
|tt0000335|nm1010955|
|tt0000335|nm1012612|
+---------+---------+
only showing top 2 rows



In [24]:
df_actors = name_basics.join(df_title, ["nconst"], 'inner')

In [25]:
df_actors.show(2)

+---------+------------+---------+
|   nconst| primaryName|   tconst|
+---------+------------+---------+
|nm0000004|John Belushi|tt0077621|
|nm0000004|John Belushi|tt0077975|
+---------+------------+---------+
only showing top 2 rows



In [27]:
df_actors = df_actors.select("tconst","primaryName")

In [29]:
df_actors.show(2)

+---------+------------+
|   tconst| primaryName|
+---------+------------+
|tt0077621|John Belushi|
|tt0077975|John Belushi|
+---------+------------+
only showing top 2 rows



In [30]:
baskets_ = df_actors.groupBy('tconst').agg(collect_set('primaryName').alias('actors'))
baskets_.createOrReplaceTempView('baskets_')
df_actors.show(5)
baskets_.show(5)

+---------+------------+
|   tconst| primaryName|
+---------+------------+
|tt0077621|John Belushi|
|tt0077975|John Belushi|
|tt0082200|John Belushi|
|tt0082801|John Belushi|
|tt0078723|John Belushi|
+---------+------------+
only showing top 5 rows

+---------+--------------------+
|   tconst|              actors|
+---------+--------------------+
|tt0000335|[John Jones, Mr. ...|
|tt0000502|[Antonio del Pozo...|
|tt0000630|[Fernanda Negri P...|
|tt0000676|[Jaime Borrás, Jo...|
|tt0000793|      [Henny Porten]|
+---------+--------------------+
only showing top 5 rows



In [31]:
fpGrowth_ = FPGrowth(itemsCol="actors", minSupport=0.00001, minConfidence=0.00001)
model_ = fpGrowth_.fit(baskets_)

In [32]:
# Display frequent itemsets.
mostPopularItemInABasket_ = model_.freqItemsets
mostPopularItemInABasket_.createOrReplaceTempView("mostPopularItemInABasket_")
mostPopularItemInABasket_.show()

# Display generated association rules.
associationRules_ = model_.associationRules
associationRules_.createOrReplaceTempView("associationRules_")
associationRules_.show()

# transform examines the input items against all the association rules and summarize the consequents as prediction
associations_ = model_.transform(baskets_)
associations_.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+----+
|               items|freq|
+--------------------+----+
|   [Leilani Sarelle]|   7|
|        [Ayla Algan]|  20|
|      [Trevor Moore]|   4|
|[Baburao Pendharkar]|  46|
|[Baburao Pendhark...|   8|
|           [Wu Yung]|  10|
|[Wu Yung, Chi-Sin...|   4|
|[Wu Yung, Man-Lei...|   4|
|[Wu Yung, Cho-Cho...|   4|
|     [Ruth Donnelly]|  31|
|[Ruth Donnelly, R...|   4|
|      [Raf Mattioli]|   6|
|       [Juan Peláez]|  34|
|[Juan Peláez, Jor...|   5|
|[Juan Peláez, Ser...|   4|
|[Juan Peláez, Bru...|   4|
|     [Chona Delgado]|   9|
|[Chona Delgado, D...|   5|
|[Chona Delgado, L...|   4|
|[Chona Delgado, L...|   4|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+------------

In [33]:
query_ = """select items, freq
           from mostPopularItemInABasket_
           where size(items) > 5
           order by freq desc
           limit 20"""
spark.sql(query_).show(5)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Rihaku Arashi, C...|  20|
|[Kakumatsuro Aras...|  17|
|[Chosei Kataoka, ...|  17|
|[Sentarô Nakamura...|  16|
|[Bradford Hill, J...|  15|
+--------------------+----+
only showing top 5 rows



In [34]:
query_ = """select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
           from associationRules_
           order by confidence desc
           limit 20"""

spark.sql(query_).show(10)

+--------------------+--------------------+----------+
|     antecedent (if)|   consequent (then)|confidence|
+--------------------+--------------------+----------+
|      [Rafael Solis]|       [Luis Huizar]|       1.0|
|[Penny Edwards, T...|        [Roy Rogers]|       1.0|
|[Rolly Quizon, De...|       [Nida Blanca]|       1.0|
|   [Marjorie Benson]|     [Violet Hopson]|       1.0|
|[Rolly Quizon, De...|            [Dolphy]|       1.0|
|[Angela Perkins, ...|[April Melody Rox...|       1.0|
|[Utako Suzuki, Ky...|  [Kôichi Katsuragi]|       1.0|
|[Bradford Hill, M...|     [David Gerrold]|       1.0|
|[Keisha, Rick Sav...|   [Caroline Laurie]|       1.0|
|[Matutina, Dely A...|      [Rolly Quizon]|       1.0|
+--------------------+--------------------+----------+
only showing top 10 rows



In [26]:
#from pyspark.sql import functions as F
#transactions= name_basics.groupby("nconst").agg(F.collect_list("primaryName").alias('actor_name'), F.collect_list("nconst").alias('actor_id'))
#transactions.show()